### Load data

In [1]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_train_valid -= 769
y_test -= 769


print(np.unique(y_train_valid))
print(np.unique(y_test))

[0 1 2 3]
[0 1 2 3]


### Shape of data

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Prepare Training Data

In [3]:
import torch
from torch.utils import data

X_train_valid = torch.from_numpy(X_train_valid).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
train_data = data.TensorDataset(X_train_valid, y_train_valid)

person_train_valid = torch.from_numpy(person_train_valid)
person_test = torch.from_numpy(person_test)

In [4]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 128
valid_size = 0.2

"""
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
"""

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)

## CNN Architecture

### AlexNet (modified)

In [5]:
import torch

train_on_gpu = torch.cuda.is_available()

# check if CUDA is available
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)
if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

cuda:0
Training on GPU ...


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):

    def __init__(self, num_classes=4):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(22, 64, kernel_size=22, stride=2),
            nn.BatchNorm1d(64, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=12, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(64, 192, kernel_size=12),
            nn.BatchNorm1d(192, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Dropout(p=0.8),
            nn.Conv1d(192, 384, kernel_size=4, stride=2),
            nn.BatchNorm1d(384, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.8),
            nn.Conv1d(384, 256, kernel_size=4, stride=2),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.8),
            nn.Conv1d(256, 256, kernel_size=4),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=1),
            
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256 * 21, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# create a complete CNN
model = AlexNet()

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

### Train the Model

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.0005, lr=1e-3)

#valid_loss_min = np.Inf # track change in validation loss
val_acc_max = 0

In [8]:
# number of epochs to train the model
n_epochs = 2000

#valid_loss_min = 0.784589

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    train_correct, train_total = 0, 0
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        #print(target.data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        _, predicted = torch.max(output, 1)
        train_total += len(target)
        train_correct += (predicted == target).sum().item()
    train_acc = train_correct/train_total
    ######################    
    # validate the model #
    ######################
    model.eval()
    val_correct, val_total = 0, 0
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
        _, predicted = torch.max(output, 1)
        val_total += len(target)
        val_correct += (predicted == target).sum().item()
    val_acc = val_correct/val_total
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print('Epoch{}  Train Loss: {:.4f}  Val Loss: {:.4f}  Train Acc: {:.6f} Val Acc: {:.6f}'.format(
        epoch, train_loss, valid_loss, train_acc, val_acc))
    
    # save model if validation loss has decreased
    if val_acc >= val_acc_max:
        print('Validation acc increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        val_acc_max,
        val_acc))
        torch.save(model.state_dict(), 'model_EEG.pt')
        val_acc_max = val_acc

 Train Loss: 0.3566  Val Loss: 1.0907  Train Acc: 0.854019 Val Acc: 0.557920
Epoch1772  Train Loss: 0.3655  Val Loss: 1.1971  Train Acc: 0.864657 Val Acc: 0.508274
Epoch1773  Train Loss: 0.3790  Val Loss: 1.0986  Train Acc: 0.849882 Val Acc: 0.550827
Epoch1774  Train Loss: 0.3926  Val Loss: 1.0914  Train Acc: 0.851655 Val Acc: 0.569740
Epoch1775  Train Loss: 0.4199  Val Loss: 1.2001  Train Acc: 0.841017 Val Acc: 0.513002
Epoch1776  Train Loss: 0.3871  Val Loss: 1.1302  Train Acc: 0.848109 Val Acc: 0.553191
Epoch1777  Train Loss: 0.3738  Val Loss: 1.2464  Train Acc: 0.854610 Val Acc: 0.482270
Epoch1778  Train Loss: 0.3525  Val Loss: 1.0053  Train Acc: 0.858156 Val Acc: 0.593381
Epoch1779  Train Loss: 0.3636  Val Loss: 1.3266  Train Acc: 0.851064 Val Acc: 0.486998
Epoch1780  Train Loss: 0.3781  Val Loss: 0.9999  Train Acc: 0.850473 Val Acc: 0.602837
Epoch1781  Train Loss: 0.3894  Val Loss: 1.1355  Train Acc: 0.849882 Val Acc: 0.534279
Epoch1782  Train Loss: 0.3666  Val Loss: 1.1200  Trai

### Load the Model with the Lowest Validation Loss

In [12]:
model.load_state_dict(torch.load('model_EEG.pt'))

<All keys matched successfully>

### Test the Trained Network

In [13]:
from torch.utils import data

batch_size = 200

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()
test_data = data.TensorDataset(X_test, y_test)

test_loader = data.DataLoader(test_data, batch_size=batch_size)

TypeError: expected np.ndarray (got Tensor)

In [14]:
# specify the target classes
classes = [769, 770, 771, 772]

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(correct.shape[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.968028

Test Accuracy of   769: 37% (42/111)
Test Accuracy of   770: 77% (98/127)
Test Accuracy of   771: 71% (69/96)
Test Accuracy of   772: 55% (61/109)

Test Accuracy (Overall): 60% (270/443)
